# CFFI

* [CFFI documentation — CFFI 1.15.1 documentation](https://cffi.readthedocs.io/en/latest/index.html)
* [python - How to pass a Numpy array into a cffi function and how to get one back out? - Stack Overflow](https://stackoverflow.com/questions/16276268/how-to-pass-a-numpy-array-into-a-cffi-function-and-how-to-get-one-back-out)

## allocate arrays as numpy ndarray

In [8]:
import numpy as np

In [9]:
x = np.random.rand(10**6)
y = np.random.rand(10**6)
z = np.zeros_like(x)

In [10]:
%%time
for i in range(0,len(x)):
    z[i] = x[i] + 3.14*y[i]

CPU times: user 308 ms, sys: 3.35 ms, total: 312 ms
Wall time: 311 ms


In [11]:
%%time
z = x+3.14*y

CPU times: user 1.08 ms, sys: 2.06 ms, total: 3.14 ms
Wall time: 2.64 ms


## define `triad()` in C

In [12]:
from cffi import FFI
build_ffi = FFI()

## build and load

In [13]:
build_ffi.cdef("""
    int triad(double*, double*, double*, int);
""")

build_ffi.set_source("_triad",
    r"""int triad(double *px, double *py, double *pz, int n) {
            #pragma omp parallel for
            for (int i = 0; i  < n; i++) pz[i] = px[i] + 3.14*py[i];
            return 0;
        }
     """)

In [14]:
lib_path = build_ffi.compile(verbose=True)

generating ./_triad.c
(already up-to-date)
the current directory is '/home/manabu/SX-Aurora_TSUBASA/Python'
running build_ext
building '_triad' extension
gcc -pthread -B /opt/anaconda3/envs/jupyter/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/anaconda3/envs/jupyter/include -fPIC -O2 -isystem /opt/anaconda3/envs/jupyter/include -fPIC -I/opt/anaconda3/envs/jupyter/include/python3.8 -c _triad.c -o ./_triad.o
gcc -pthread -B /opt/anaconda3/envs/jupyter/compiler_compat -shared -Wl,--allow-shlib-undefined -Wl,-rpath,/opt/anaconda3/envs/jupyter/lib -Wl,-rpath-link,/opt/anaconda3/envs/jupyter/lib -L/opt/anaconda3/envs/jupyter/lib -Wl,--allow-shlib-undefined -Wl,-rpath,/opt/anaconda3/envs/jupyter/lib -Wl,-rpath-link,/opt/anaconda3/envs/jupyter/lib -L/opt/anaconda3/envs/jupyter/lib ./_triad.o -o ./_triad.cpython-38-x86_64-linux-gnu.so


* `import`可能なCPython Shared Objectができる

In [15]:
lib_path

'/home/manabu/SX-Aurora_TSUBASA/Python/_triad.cpython-38-x86_64-linux-gnu.so'

In [16]:
!nm $lib_path

                 U PyArg_UnpackTuple
                 U PyErr_Occurred
                 U PyEval_RestoreThread
                 U PyEval_SaveThread
                 U PyImport_ImportModule
00000000000014c0 T PyInit__triad
                 U PyLong_FromLong
                 U PyLong_FromVoidPtr
                 U PyObject_CallMethod
                 U PyObject_Free
                 U PyObject_Malloc
0000000000003e00 d _DYNAMIC
0000000000004000 d _GLOBAL_OFFSET_TABLE_
                 w _ITM_deregisterTMCloneTable
                 w _ITM_registerTMCloneTable
                 U _Py_Dealloc
00000000000021c4 r __FRAME_END__
0000000000002040 r __GNU_EH_FRAME_HDR
00000000000040b8 d __TMC_END__
                 w __cxa_finalize@GLIBC_2.2.5
0000000000001180 t __do_global_dtors_aux
0000000000003d58 t __do_global_dtors_aux_fini_array_entry
0000000000003d60 d __dso_handle
0000000000003d50 t __frame_dummy_init_array_entry
                 w __gmon_start__
00000000000011d0 t _cffi_convert_array_argu

In [17]:
#lib = ffi.dlopen(lib_path)

## load

In [18]:
from _triad import lib, ffi

In [22]:
px = ffi.cast("double *", x.ctypes.data)
py = ffi.cast("double *", y.ctypes.data)
#pz = ffi.cast("double *", z.ctypes.data)

In [23]:
px, py

(<cdata 'double *' 0x55cab3801210>, <cdata 'double *' 0x55cab3fa2420>)

## benchmark

In [24]:
z = np.zeros_like(x)
pz = ffi.cast("double *", z.ctypes.data)
pz

<cdata 'double *' 0x55cab4ee4840>

In [25]:
lib.triad

<function _triad.Lib.triad>

In [26]:
%%time
lib.triad(px, py, pz, len(z))

CPU times: user 2.28 ms, sys: 0 ns, total: 2.28 ms
Wall time: 1.8 ms


0

In [27]:
pz[3], px[3]+3.14*py[3]

(0.48085690670269754, 0.48085690670269754)

In [14]:
import _imp

In [15]:
_imp.extension_suffixes()

['.cpython-38-x86_64-linux-gnu.so', '.abi3.so', '.so']

* [Python/CFFIを利用してC拡張を作成する (1/2) - 2018-01-17 - ククログ](https://www.clear-code.com/blog/2018/1/17.html)
* [Python/CFFIを利用してC拡張を作成する (2/2) - 2018-04-05 - ククログ](https://www.clear-code.com/blog/2018/4/5.html)